# Achate a curva
> Número de casos ativos e estimativa de leitos de UTI total e para pacientes covid-19

- toc: true
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [jupyter]
- image: images/brazil-flatten-the-curve.png

In [1]:
#hide
import pandas as pd
import altair as alt
import math
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

/usr/local/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#hide
STATE_COLUMN = "Estado"
DATE_COLUMN = "Data"
CONFIRMED_CASES = 'Casos confirmados'
DEATHS = 'Mortes'
ACTIVE_COLUMN = "Casos Ativos"
VALUE = "Valor"
CASES_NEED_ICU = 0.12
POPULATION = 211000000
ICU_PER_100k = 20
TOTAL_ICU = (211000000 / 100000 * 20) + 1456
AVAILABLE_PERCENTAGE_ICU = 0.3
TOTAL_AVAILABLE_ICU = TOTAL_ICU * AVAILABLE_PERCENTAGE_ICU
CASES_NEED_ICU_COLUMN = "Precisam de UTI"
CASES_DONT_NEED_ICU_COLUMN = "Sem necessidade de UTI"
PATIENTS = "Pacientes"

In [3]:
#hide
url_cases = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
url_deaths = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
url_recoveries = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

src = pd.read_csv(url_cases)
df_cases = src[(src['Country/Region'] == 'Brazil')]
df_cases['base'] = 'cases'

src = pd.read_csv(url_recoveries)
df_recoveries = src[(src['Country/Region'] == 'Brazil')]
df_recoveries['base'] = 'recoveries'

src = pd.read_csv(url_deaths)
df_deaths = src[(src['Country/Region'] == 'Brazil')]
df_deaths['base'] = 'deaths'

df = [df_cases, df_recoveries, df_deaths]
df = pd.concat(df)

df = df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1)
#df = df.reset_index()

# Add active cases row
df = df.set_index('base')
new_row = df.loc['cases'] - df.loc['recoveries'] - df.loc['deaths']
new_row.name = 'actives'
df = df.append([new_row])
# Reset index after added new row
df = df.reset_index()
df = df.rename(columns={"index":"base"})

# Melt structure (unpivot)
dt_cols = list(df.columns[~df.columns.isin(['base','index'])])
df = df.melt(id_vars=['base'], value_vars=dt_cols)
df = df.rename(columns={
     "base": STATE_COLUMN, 
     "variable":DATE_COLUMN, 
     "value":VALUE,
     "actives": ACTIVE_COLUMN
})
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')
df = df.sort_values(by=[DATE_COLUMN])

df = df[df[STATE_COLUMN].isin(['actives'])]

data = df.copy()
#df.tail()

<ipython-input-3-39af19bb8182>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cases['base'] = 'cases'
<ipython-input-3-39af19bb8182>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recoveries['base'] = 'recoveries'
<ipython-input-3-39af19bb8182>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [4]:
#hide
df = data.copy()
STATE_COLUMN = PATIENTS
df[VALUE] = round(df[VALUE] * (CASES_NEED_ICU),0)
df[STATE_COLUMN] = CASES_NEED_ICU_COLUMN
df = df.rename(columns={VALUE:ACTIVE_COLUMN})

In [5]:
#hide_input
selection = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(ACTIVE_COLUMN, axis=alt.Axis(title="{:.0f}".format(CASES_NEED_ICU*100)+"% dos "+ACTIVE_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, ACTIVE_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Achate a curva - apenas os ativos que estima-se que precisem de UTI",
        " * ver premissas"
    ]
).add_selection(
    selection
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("UTIs para pacientes COVID19: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU))
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 | legend

alt.HConcatChart(...)

In [6]:
#hide
df1 = data.copy()
STATE_COLUMN = PATIENTS
df1[VALUE] = round(df1[VALUE] * (CASES_NEED_ICU),0)
df1[STATE_COLUMN] = CASES_NEED_ICU_COLUMN
df2 = data.copy()
df2[VALUE] = round(df2[VALUE] * (1-CASES_NEED_ICU),)
df2[STATE_COLUMN] = CASES_DONT_NEED_ICU_COLUMN
df = pd.concat([df1, df2])
df = df.rename(columns={VALUE:ACTIVE_COLUMN})

In [7]:
#hide_input
selection = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(ACTIVE_COLUMN, axis=alt.Axis(title=ACTIVE_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, ACTIVE_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Achate a curva - todos os casos ativos",
        " * ver premissas"
    ]
).add_selection(
    selection
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("UTIs para pacientes COVID19: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU))
)

x2line = alt.Chart(pd.DataFrame({'y': [TOTAL_ICU]})).mark_rule(color='darkred', strokeWidth=2).encode(
    y='y:Q'
)
text2 = x2line.mark_text(align='left', x=5, dy=10, color='darkred', strokeWidth=1).encode(
    text=alt.value("Total de UTIs: "+"{:.0f}".format(TOTAL_ICU))
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 + x2line + text2 | legend

alt.HConcatChart(...)

In [8]:
#hide_input
print("Premissas :")
print("- A população brasileira estimada é de "+"{:.0f}".format(POPULATION/1000000)+" milhões de habitantes (fonte: ibge)")
print("- O Brasil tem cerca de "+str(ICU_PER_100k)+" leitos de UTI para cada 100 mil habitantes (fonte: bcc). Outros leitos foram construídos para combater a covid-19 (fonte: ministério da saúde)")
print("- Baseado nos dados acima, estima-se que haja "+"{:.0f}".format(TOTAL_ICU)+" UTIs no país.")
print("- {:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% de UTIs disponíveis para pacientes com covid19 (fonte: artigo oglobo)")
print("- Cerca de "+"{:.0f}".format(CASES_NEED_ICU*100)+"% dos pacientes com covid-19 precisam de tratamento intensivo (fonte: artigo the lancet)")

Premissas :
- A população brasileira estimada é de 211 milhões de habitantes (fonte: ibge)
- O Brasil tem cerca de 20 leitos de UTI para cada 100 mil habitantes (fonte: bcc). Outros leitos foram construídos para combater a covid-19 (fonte: ministério da saúde)
- Baseado nos dados acima, estima-se que haja 43656 UTIs no país.
- 30% de UTIs disponíveis para pacientes com covid19 (fonte: artigo oglobo)
- Cerca de 12% dos pacientes com covid-19 precisam de tratamento intensivo (fonte: artigo the lancet)


Based on the work of [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html), adapted by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). 
Dados:
- covid19: [brasil.io](https://brasil.io/home/)
- população estimada: [ibge](https://www.ibge.gov.br/apps/populacao/projecao//)
- quantidade de leitos: [bcc](https://www.bbc.com/portuguese/brasil-52137553) e [ministério da saúde](https://www.saude.gov.br/noticias/agencia-saude/46772-brasil-ganha-reforco-de-1-134-leitos-de-uti-no-combate-ao-coronavirus)
- leitos disponíveis: [artigo oglobo](https://oglobo.globo.com/sociedade/coronavirus/coronavirus-ministerio-estima-que-sus-tem-de-12-13-mil-leitos-de-uti-disponiveis-para-atender-pacientes-1-24328523)
- pacientes que precisam de UTI: [artigo the lancet](https://www.thelancet.com/journals/lanres/article/PIIS2213-2600(20)30161-2/fulltext)